# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_results_file_path = "../output_data/cities.csv"
csv_reader = pd.read_csv(weather_results_file_path)
initial_df = pd.DataFrame(csv_reader)
initial_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Ancud,-41.8697,-73.8203,62.33,80,61,2.37,CL,1643067427
1,1,Aguazul,5.1728,-72.5471,82.13,50,79,1.87,CO,1643067429
2,2,Hobart,-42.8794,147.3294,69.53,84,75,0.45,AU,1643067431
3,3,Ushuaia,-54.8000,-68.3000,40.73,87,100,14.40,AR,1643067233
4,4,Rikitea,-23.1203,-134.9692,76.73,71,100,7.86,PF,1643067433


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(gkey)

In [4]:
locations = initial_df[["Lat", "Lng"]]
weights = initial_df["Humidity"]
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout, center = (0,0), zoom_level = 1)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
#narrow to between 75 and 90
# wind speed less then 10 and humidity less then 20
# pass in what needs to be mapped to the new dict
new_dict = {"City": [], "Country": [],"Date": [], "Lat": [], "Lng": []}

# drop any rows with nulls
initial_df.dropna()

# could potentially do with loc as well
for i in initial_df.iterrows():
    #print(i[1])
    if (int(i[1]["Max Temp"]) > 75) & (int(i[1]["Max Temp"]) < 90) & (int(i[1]["Wind Speed"]) < 10) & (int(i[1]["Humidity"] < 40)):
        new_dict["City"].append(i[1]["City"])
        new_dict["Country"].append(i[1]["Country"])
        new_dict["Date"].append(i[1]["Date"])
        new_dict["Lat"].append(i[1]["Lat"])
        new_dict["Lng"].append(i[1]["Lng"])

hotel_df = pd.DataFrame(new_dict)
new_dict

{'City': ['Yaan',
  'Codrington',
  'Rawson',
  'Savalou',
  'Podor',
  'Koundara',
  'Kapoeta'],
 'Country': ['NG', 'AU', 'AR', 'BJ', 'MR', 'GN', 'SS'],
 'Date': [1643067600,
  1643067687,
  1643067831,
  1643068010,
  1643068148,
  1643068249,
  1643068258],
 'Lat': [7.3833, -38.2667, -43.3002, 7.9281, 16.6527, 12.4833, 4.7717],
 'Lng': [8.5667, 141.9667, -65.1023, 1.9756, -14.9587, -13.3, 33.5903]}

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Date,Lat,Lng,Hotel Name
0,Yaan,NG,1643067600,7.3833,8.5667,
1,Codrington,AU,1643067687,-38.2667,141.9667,
2,Rawson,AR,1643067831,-43.3002,-65.1023,
3,Savalou,BJ,1643068010,7.9281,1.9756,
4,Podor,MR,1643068148,16.6527,-14.9587,
5,Koundara,GN,1643068249,12.4833,-13.3000,
6,Kapoeta,SS,1643068258,4.7717,33.5903,


In [7]:
marker_coordinates = []
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    target_coordinates = str(lat) + ", " + str(lng)
    target_search = "hotel"
    target_radius = 5000


# set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "key": gkey
    }
    try:
        response = requests.get(base_url, params=params)
        places_data = response.json()
        hotel_df["Hotel Name"][index] = places_data["results"][0]["name"]
        lat_coord = places_data["results"][0]["geometry"]["location"]["lat"]
        lng_coord = places_data["results"][0]["geometry"]["location"]["lng"]
        coord_tuple = (lat_coord, lng_coord)
        marker_coordinates.append(coord_tuple)
        
    except IndexError:
        hotel_df["Hotel Name"][index] = None
        lat_coord = lat
        lng_coord = lng
        coord_tuple = (lat_coord, lng_coord)
        marker_coordinates.append(coord_tuple)
    
# could reformat cities names to be upper case later
markers = gmaps.marker_layer(marker_coordinates)
fig.add_layer(markers)
hotel_df

,City,Country,Date,Lat,Lng,Hotel Name
0,Yaan,NG,1643067600,7.3833,8.5667,None
1,Codrington,AU,1643067687,-38.2667,141.9667,Codrington Gardens Bed And Breakfast
2,Rawson,AR,1643067831,-43.3002,-65.1023,Hosteria Sampedro
3,Savalou,BJ,1643068010,7.9281,1.9756,Hôtel LE RECONFORT
4,Podor,MR,1643068148,16.6527,-14.9587,Auberge Du Tékrour
5,Koundara,GN,1643068249,12.4833,-13.3000,None
6,Kapoeta,SS,1643068258,4.7717,33.5903,EASTGATE Hotel


In [8]:
# check coords
marker_coordinates

[(7.3833, 8.5667),
 (-38.2846732, 142.0097987),
 (-43.301509, -65.104922),
 (7.939984699999998, 1.9857664),
 (16.6547218, -14.9556862),
 (12.4833, -13.3),
 (4.752859, 33.598173)]

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers2 = gmaps.marker_layer(locations, info_box_content=hotel_info,display_info_box=True)
fig.add_layer(markers2)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…